In [46]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [47]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Define the SNN layer

In [48]:
class SpikingLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super(SpikingLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[input_shape[-1], self.units])

    def call(self, inputs):
        spikes = tf.cast(inputs > 0, dtype=tf.float32)
        output = tf.matmul(spikes, self.kernel)
        return output

In [49]:
model = tf.keras.Sequential([
    SpikingLayer(64),
    SpikingLayer(32),
    SpikingLayer(10)
])

In [50]:
model.compile(
    optimizer="adam", 
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
    metrics=["accuracy"]
)

model.fit(
    x_train, 
    y_train, 
    batch_size=32, 
    epochs=100, 
    validation_data=(x_test, y_test)
)

Epoch 1/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.9288 - accuracy: 0.3374 - val_loss: 1.7185 - val_accuracy: 0.4259
Epoch 2/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6721 - accuracy: 0.4284 - val_loss: 1.6435 - val_accuracy: 0.4426
Epoch 3/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6329 - accuracy: 0.4357 - val_loss: 1.6240 - val_accuracy: 0.4464
Epoch 4/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6210 - accuracy: 0.4390 - val_loss: 1.6177 - val_accuracy: 0.4499
Epoch 5/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6159 - accuracy: 0.4406 - val_loss: 1.6148 - val_accuracy: 0.4450
Epoch 6/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6133 - accuracy: 0.4418 - val_loss: 1.6102 - val_accuracy: 0.4498
Epoch 7/100
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6117 - accuracy: 0.4419 - val_loss: 1.6100 - val_ac

In [51]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 spiking_layer (SpikingLayer  (None, 64)               50176     
 )                                                               
                                                                 
 spiking_layer_1 (SpikingLay  (None, 32)               2048      
 er)                                                             
                                                                 
 spiking_layer_2 (SpikingLay  (None, 10)               320       
 er)                                                             
                                                                 
Total params: 52,544
Trainable params: 52,544
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.save("SNN_MNIST.h5")

In [53]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

313/313 [==============================] - 0s 912us/step - loss: 1.5973 - accuracy: 0.4556
Test Loss: 1.5973379611968994
Test Accuracy: 0.45559999346733093


# hls4ml Config Part

In [36]:
import hls4ml

class SNNLayer(hls4ml.model.layers.Layer):

    def initialize(self):
        inp = self.get_input_variable()
        shape = inp.shape
        dims = inp.dim_names
        self.add_output_variavle(shape, dims)

In [35]:
def parse_reverse_layer(keras_layer, input_names, input_shapes, data_reader):
    layer = {}
    layer['class_name'] = 'SNNLayer'
    layer['name'] = keras_layer['config']['name']
    layer['n_in'] = input_shapes[0][1]

    if input_names is not None:
        layer['inputs'] = input_names

    return layer, [shape for shape in input_shapes[0]]